In [1]:
import os
import sys
HOME_DIR = '/Users/ag47/Desktop/COS40007-Artificial-Intelligence-for-Engineering-main'
os.chdir(HOME_DIR)

DATASETS_DIR = f'{HOME_DIR}/datasets'
DATA_DIR = f'{DATASETS_DIR}/11'

TRAIN_CSV_FILE = f'{DATA_DIR}/bounding_boxes/train_labels.csv'
TRAIN_IMAGES_DIR = f'{DATA_DIR}/images/train'
TRAIN_LABELS_DIR = f'{DATA_DIR}/labels/train'
SELECTED_TRAIN_IMAGES_DIR= f'{DATA_DIR}/images/train_selected'
SELECTED_TRAIN_LABELS_DIR = f'{DATA_DIR}/labels/train_selected'


TEST_CSV_FILE = f'{DATA_DIR}/bounding_boxes/test_labels.csv'
TEST_IMAGES_DIR = f'{DATA_DIR}/images/test'
TEST_LABELS_DIR = f'{DATA_DIR}/labels/test'
SELECTED_TEST_IMAGES_DIR = f'{DATA_DIR}/images/test_selected'
SELECTED_TEST_LABELS_DIR = f'{DATA_DIR}/labels/test_selected'

OUTPUT_IMAGES_DIR = 'week-06-portfolio/evaluation_images'

yaml_file_path = f'{DATA_DIR}/graffiti.yaml'

print(f'HOME_DIR: {HOME_DIR}')
print(f'DATASETS_DIR: {DATASETS_DIR}')
print(f'DATA_DIR: {DATA_DIR}')
print(f'TRAIN_CSV_FILE: {TRAIN_CSV_FILE}')
print(f'TRAIN_IMAGES_DIR: {TRAIN_IMAGES_DIR}')
print(f'TRAIN_LABELS_DIR: {TRAIN_LABELS_DIR}')
print(f'TEST_CSV_FILE: {TEST_CSV_FILE}')
print(f'TEST_IMAGES_DIR: {TEST_IMAGES_DIR}')
print(f'TEST_LABELS_DIR: {TEST_LABELS_DIR}')

HOME_DIR: /Users/ag47/Desktop/COS40007-Artificial-Intelligence-for-Engineering-main
DATASETS_DIR: /Users/ag47/Desktop/COS40007-Artificial-Intelligence-for-Engineering-main/datasets
DATA_DIR: /Users/ag47/Desktop/COS40007-Artificial-Intelligence-for-Engineering-main/datasets/11
TRAIN_CSV_FILE: /Users/ag47/Desktop/COS40007-Artificial-Intelligence-for-Engineering-main/datasets/11/bounding_boxes/train_labels.csv
TRAIN_IMAGES_DIR: /Users/ag47/Desktop/COS40007-Artificial-Intelligence-for-Engineering-main/datasets/11/images/train
TRAIN_LABELS_DIR: /Users/ag47/Desktop/COS40007-Artificial-Intelligence-for-Engineering-main/datasets/11/labels/train
TEST_CSV_FILE: /Users/ag47/Desktop/COS40007-Artificial-Intelligence-for-Engineering-main/datasets/11/bounding_boxes/test_labels.csv
TEST_IMAGES_DIR: /Users/ag47/Desktop/COS40007-Artificial-Intelligence-for-Engineering-main/datasets/11/images/test
TEST_LABELS_DIR: /Users/ag47/Desktop/COS40007-Artificial-Intelligence-for-Engineering-main/datasets/11/label

In [2]:
!pip install --upgrade pip --quiet
!pip install -r requirements.txt --upgrade --quiet

In [3]:
import json
import os
import platform
import random
import re
import shutil
import subprocess
import cv2
import pandas as pd
import requests
import torch
from PIL import Image
from tqdm import tqdm
from ultralytics import YOLO
from torchvision.ops import box_iou



ModuleNotFoundError: No module named 'cv2'

In [3]:
if platform.system() == 'Darwin':
    device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: mps


In [4]:
def convert_annotations(csv_file, images_dir, output_dir, class_mapping):
    df = pd.read_csv(csv_file)
    grouped = df.groupby('filename')

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename, group in tqdm(grouped, desc=f'Converting annotations for {csv_file}'):
        image_path = os.path.join(images_dir, filename)
        if not os.path.exists(image_path):
            continue  # Skip if image does not exist

        img_width = group.iloc[0]['width']
        img_height = group.iloc[0]['height']

        annotations = []
        for _, row in group.iterrows():
            class_id = class_mapping[row['class']]
            xmin = row['xmin']
            ymin = row['ymin']
            xmax = row['xmax']
            ymax = row['ymax']

            # Convert to YOLO format
            x_center = ((xmin + xmax) / 2) / img_width
            y_center = ((ymin + ymax) / 2) / img_height
            bbox_width = (xmax - xmin) / img_width
            bbox_height = (ymax - ymin) / img_height

            annotations.append(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}")

        # Write annotations to file
        txt_filename = os.path.splitext(filename)[0] + '.txt'
        with open(os.path.join(output_dir, txt_filename), 'w') as f:
            for ann in annotations:
                f.write(ann + '\n')


In [ ]:
# Define class mapping
class_mapping = {'Graffiti': 0}

# Convert training annotations
convert_annotations(TRAIN_CSV_FILE, TRAIN_IMAGES_DIR, TRAIN_LABELS_DIR, class_mapping)

# Convert test annotations
convert_annotations(TEST_CSV_FILE, TEST_IMAGES_DIR, TEST_LABELS_DIR, class_mapping)


In [6]:
def select_random_images(source_dir, dest_dir, num_images, used_images=set()):
    images = [f for f in os.listdir(source_dir) if f.endswith('.jpg') and f not in used_images]
    selected_images = random.sample(images, min(num_images, len(images)))
    used_images.update(selected_images)

    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    
    for file in os.listdir(dest_dir):
        file_path = os.path.join(dest_dir, file)
        if os.path.isfile(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    
    for img in selected_images:
        shutil.copy(os.path.join(source_dir, img), os.path.join(dest_dir, img))
    return used_images

def copy_annotation_files(image_dir, label_source_dir, label_dest_dir):
    if not os.path.exists(label_dest_dir):
        os.makedirs(label_dest_dir)
        
    for file in os.listdir(label_dest_dir):
        file_path = os.path.join(label_dest_dir, file)
        if os.path.isfile(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    
    
    for img_file in os.listdir(image_dir):
        if img_file.endswith('.jpg'):
            base_name = os.path.splitext(img_file)[0]
            label_file = base_name + '.txt'
            src_label_path = os.path.join(label_source_dir, label_file)
            dst_label_path = os.path.join(label_dest_dir, label_file)
            if os.path.exists(src_label_path):
                shutil.copy(src_label_path, dst_label_path)

In [7]:
# Set random seed for reproducibility
random.seed(42)

# Initialize used images sets
used_train_images = set()
used_test_images = set()

# Select 400 random training images
used_train_images = select_random_images(TRAIN_IMAGES_DIR, SELECTED_TRAIN_IMAGES_DIR, 400, used_train_images)

# Copy corresponding training annotation files
copy_annotation_files(SELECTED_TRAIN_IMAGES_DIR, TRAIN_LABELS_DIR, SELECTED_TRAIN_LABELS_DIR)

# Select 40 random test images
used_test_images = select_random_images(TEST_IMAGES_DIR, SELECTED_TEST_IMAGES_DIR, 40, used_test_images)

# Copy corresponding test annotation files
copy_annotation_files(SELECTED_TEST_IMAGES_DIR, TEST_LABELS_DIR, SELECTED_TEST_LABELS_DIR)

In [8]:
def create_yaml_file(file_path, train_images, val_images, nc, class_names):
    data = {
        'train': train_images,
        'val': val_images,
        'nc': nc,
        'names': class_names
    }
    with open(file_path, 'w') as f:
        for key, value in data.items():
            if isinstance(value, list):
                value_str = str(value).replace("'", "")
                f.write(f"{key}: {value_str}\n")
            else:
                f.write(f"{key}: {value}\n")

In [9]:
# Usage
train_images_path = os.path.abspath(SELECTED_TRAIN_IMAGES_DIR)
val_images_path = os.path.abspath(SELECTED_TEST_IMAGES_DIR)  # Using test data as validation set

nc = 1
class_names = ['Graffiti']

create_yaml_file(yaml_file_path, train_images_path, val_images_path, nc, class_names)

# Test Run

In [ ]:
# Load a pretrained YOLOv5s model
model = YOLO('week-06-portfolio/models/yolov5su.pt')

# Train the model
results = model.train(data=yaml_file_path, epochs=1, imgsz=640, batch=16, name='graffiti_detection', device=device)

In [12]:
def compute_iou(pred_boxes, true_boxes):
    # Convert boxes to tensors
    pred_boxes = torch.tensor(pred_boxes)
    true_boxes = torch.tensor(true_boxes)

    # Compute IoU
    iou = box_iou(pred_boxes, true_boxes)
    return iou.diag().numpy()  # Get IoUs for matched boxes

def evaluate_model(model, images_dir, labels_dir, output_images_dir=None):
    results = []
    images = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]

    if output_images_dir and not os.path.exists(output_images_dir):
        os.makedirs(output_images_dir)

    for img_name in tqdm(images, desc='Evaluating model'):
        img_path = os.path.join(images_dir, img_name)
        label_path = os.path.join(labels_dir, os.path.splitext(img_name)[0] + '.txt')

        # Perform inference
        preds = model.predict(img_path, conf=0.25)

        # Load image for drawing
        img = cv2.imread(img_path)
        img_height, img_width = img.shape[:2]

        # Get predicted boxes and confidence scores
        pred_boxes = []
        confidences = []
        for pred in preds:
            for box in pred.boxes:
                x_min = box.xyxy[0][0].item()
                y_min = box.xyxy[0][1].item()
                x_max = box.xyxy[0][2].item()
                y_max = box.xyxy[0][3].item()
                conf = box.conf.item()
                pred_boxes.append([x_min, y_min, x_max, y_max])
                confidences.append(conf)

                # Draw predicted bounding box
                if output_images_dir:
                    label = f"{model.names[int(box.cls)]}: {conf:.2f}"
                    cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 2)
                    cv2.putText(img, label, (int(x_min), int(y_min) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Get true boxes
        true_boxes = []
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    class_id, x_center, y_center, width, height = map(float, line.strip().split())
                    # Convert back to absolute coordinates
                    x_center *= img_width
                    y_center *= img_height
                    width *= img_width
                    height *= img_height
                    x_min = x_center - width / 2
                    y_min = y_center - height / 2
                    x_max = x_center + width / 2
                    y_max = y_center + height / 2
                    true_boxes.append([x_min, y_min, x_max, y_max])

                    # Draw true bounding box
                    if output_images_dir:
                        cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 0, 255), 2)

        # Save image with drawn bounding boxes
        if output_images_dir:
            output_image_path = os.path.join(output_images_dir, img_name)
            cv2.imwrite(output_image_path, img)

        # Compute IoU
        if pred_boxes and true_boxes:
            ious = compute_iou(pred_boxes, true_boxes)
            max_iou = max(ious)
            max_conf = confidences[ious.argmax()]
        else:
            max_iou = 0.0
            max_conf = 0.0 if not confidences else max(confidences)

        results.append({
            'image_name': img_name,
            'confidence_value': max_conf,
            'IoU_value': max_iou
        })

    return pd.DataFrame(results)


In [5]:
# Usage
df_results = evaluate_model(model, SELECTED_TEST_IMAGES_DIR, SELECTED_TEST_LABELS_DIR, OUTPUT_IMAGES_DIR)
df_results.to_csv('week-06-portfolio/evaluation_results.csv', index=False)

NameError: name 'evaluate_model' is not defined

# Full Train Base on the Requirements

In [ ]:
# Initialize variables for iterative training
iou_threshold = 0.9
satisfied = False
iteration = 1

while not satisfied:
    print(f"\nStarting iteration {iteration}")

    # Select new training images
    training_images = random.sample(os.listdir(TRAIN_IMAGES_DIR), 400)

    # Update training data directories
    selected_train_images_dir = f'week-06-portfolio/images/train_selected_iter_{iteration}'
    selected_train_labels_dir = f'week-06-portfolio/labels/train_selected_iter_{iteration}'
    if not os.path.exists(selected_train_images_dir):
        os.makedirs(selected_train_images_dir)
    if not os.path.exists(selected_train_labels_dir):
        os.makedirs(selected_train_labels_dir)

    # Copy selected images and annotations
    for img in training_images:
        shutil.copy(os.path.join(TRAIN_IMAGES_DIR, img), os.path.join(selected_train_images_dir, img))
        label_file = os.path.splitext(img)[0] + '.txt'
        src_label_path = os.path.join(TRAIN_LABELS_DIR, label_file)
        dst_label_path = os.path.join(selected_train_labels_dir, label_file)
        if os.path.exists(src_label_path):
            shutil.copy(src_label_path, dst_label_path)

    # Update YAML file
    train_images_path = os.path.abspath(selected_train_images_dir)
    val_images_path = os.path.abspath(SELECTED_TEST_IMAGES_DIR)

    # Update the YAML file path for this iteration
    yaml_dir = 'week-06-portfolio/yaml'
    if not os.path.exists(yaml_dir):
        os.makedirs(yaml_dir)
        
    yaml_file_path_iter = f'week-06-portfolio/yaml/graffiti_iter_{iteration}.yaml'
    create_yaml_file(yaml_file_path_iter, train_images_path, val_images_path, nc, class_names)

    # Load the model from previous iteration
    if iteration == 1:
        model = YOLO('week-06-portfolio/models/yolov5su.pt')  # Start with pre-trained YOLOv5su model
    else:
        previous_model_path = f'week-06-portfolio/runs/train/graffiti_detection_iter_{iteration - 1}/weights/best.pt'
        model = YOLO(previous_model_path)

    # Train the model with 5 epochs
    model.train(
        data=yaml_file_path_iter,
        epochs=5,
        imgsz=640,
        batch=16,
        project='week-06-portfolio/runs/train',
        name=f'graffiti_detection_iter_{iteration}',
        device=device
    )

    # Evaluate the model
    output_images_dir_iter = f'week-06-portfolio/evaluation_images_iter_{iteration}'
    if not os.path.exists(output_images_dir_iter):
        os.makedirs(output_images_dir_iter)

    df_results = evaluate_model(model, SELECTED_TEST_IMAGES_DIR, SELECTED_TEST_LABELS_DIR, output_images_dir_iter)
    df_results.to_csv(f'week-06-portfolio/evaluation_results_iter_{iteration}.csv', index=False)

    # Check IoU threshold
    over_threshold = df_results[df_results['IoU_value'] > iou_threshold]
    if len(over_threshold) / len(df_results) >= 0.8:
        print(f"IoU threshold met in iteration {iteration}")
        model.save(f'week-06-portfolio/models/yolov5s_graffiti_iter_{iteration}.pt')
        satisfied = True
    else:
        print(f"IoU threshold not met in iteration {iteration}")

    # Prepare for next iteration
    iteration += 1


if not satisfied:
    print("IoU threshold not met after all iterations.")
else:
    print("Training completed successfully.")



In [4]:
def move_items_and_folders(source_dir, target_dir, exclude_dirs):

    # Ensure the source directory exists
    if not os.path.exists(source_dir):
        print(f"Error: Source directory does not exist: {source_dir}")
        return

    # Create the target directory if it does not exist
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
        print(f"Created target directory: {target_dir}")
    else:
        print(f"Target directory already exists: {target_dir}")

    # Iterate over all items in the source directory
    for item in os.listdir(source_dir):
        item_path = os.path.join(source_dir, item)

        # Skip the target directory itself to prevent recursive moving
        if os.path.abspath(item_path) == os.path.abspath(target_dir):
            print(f"Skipped target directory itself: '{item}'")
            continue

        # Check if the item is a directory
        if os.path.isdir(item_path):
            if item in exclude_dirs:
                print(f"Excluded directory: '{item}'.")
                continue  # Skip excluded directories

            # Define the destination path
            dest_path = os.path.join(target_dir, item)

            # Check if destination already exists
            if os.path.exists(dest_path):
                print(f"Skipped: Destination already exists for folder '{item}'.")
                continue

            try:
                # Move the directory
                shutil.move(item_path, dest_path)
                print(f"Moved folder '{item}' to '{target_dir}'.")
            except Exception as e:
                print(f"Error moving folder '{item}': {e}")

        elif os.path.isfile(item_path):
            # If the item is a file, move it to the target directory
            dest_path = os.path.join(target_dir, item)

            # Check if destination file already exists
            if os.path.exists(dest_path):
                print(f"Skipped: Destination already exists for file '{item}'.")
                continue

            try:
                shutil.move(item_path, dest_path)
                print(f"Moved file '{item}' to '{target_dir}'.")
            except Exception as e:
                print(f"Error moving file '{item}': {e}")

        else:
            # For any other types of items (e.g., symbolic links), skip them
            print(f"Skipped non-directory, non-file item: '{item}'.")

In [4]:
WEEK_6_DIR = '/Users/ag47/Desktop/COS40007-Artificial-Intelligence-for-Engineering-main/week-06-portfolio'
TARGET_TRAIN_DIR = os.path.join(WEEK_6_DIR, 'train')
EXCLUDE_DIRS = ['code', 'docs', 'requirements', 'models']
move_items_and_folders(WEEK_6_DIR, TARGET_TRAIN_DIR, EXCLUDE_DIRS)

NameError: name 'move_items_and_folders' is not defined

# Test Run on URL

In [13]:
model = YOLO('/Users/ag47/Desktop/COS40007-Artificial-Intelligence-for-Engineering-main/week-06-portfolio/train/runs/train/graffiti_detection_iter_30/weights/best.pt')

In [ ]:
# Test Run
source = 'https://videos.pexels.com/video-files/4543511/4543511-hd_1080_1920_25fps.mp4'
results = model.track(source, save=True, project=f'{HOME_DIR}/week-06-portfolio/results', tracker="bytetrack.yaml")

In [54]:
def extract_video_id(url):
    pattern = r'-([\d]+)/$'
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    else:
        return None
    
def get_video_name(video_url):
    pattern = r'/([\d]+-[\w]+_[\d]+_[\d]+_[\d]+fps\.mp4)$'
    match = re.search(pattern, video_url)
    if match:
        return match.group(1)
    else:
        return None


In [56]:
# PEXELS API DOC
# https://www.pexels.com/api/documentation/

# TUTOR MAY WANT TO REPLACE WITH YOUR OWN API KEY!
PEXELS_API = '3W6w2EgXgVBeG8j469C9YuOMLvi6xMPeFgbyd8w72vm0GjnCKuqxJzUZ'

# Function to get the HD video link
def get_hd_video_link(video_id):
    url = f"https://api.pexels.com/videos/videos/{video_id}"
    command = f'curl -H "Authorization: {PEXELS_API}" {url}'
    response = subprocess.run(command, shell=True, capture_output=True, text=True)
    
    try:
        data = json.loads(response.stdout)
        # Look for the hd link in video_files
        for video_file in data['video_files']:
            if video_file['quality'] == 'hd':
                return video_file['link']
    except json.JSONDecodeError as e:
        print(f"Failed to retrieve video data for ID: {video_id}")
        return None

In [57]:
# List of URLs
urls = [
    "https://www.pexels.com/video/a-door-with-graffiti-on-it-is-shown-4543511/",
    "https://www.pexels.com/video/busy-street-footage-854181/",
    "https://www.pexels.com/video/graffiti-painted-on-the-train-station-wall-3413463/",
    "https://www.pexels.com/video/a-man-writing-on-a-wall-with-a-marker-9724130/"
]

# Extract and print HD video links for each URL
for url in urls:
    video_id = extract_video_id(url)
    if video_id:
        hd_link = get_hd_video_link(video_id)
        if hd_link:
            print(f"{hd_link}")
            print(f'{get_video_name(hd_link)}')
    else:
        print(f"Could not extract video ID from URL: {url}")


https://videos.pexels.com/video-files/4543511/4543511-hd_720_1280_50fps.mp4
4543511-hd_720_1280_50fps.mp4
https://videos.pexels.com/video-files/854181/854181-hd_1920_1080_25fps.mp4
854181-hd_1920_1080_25fps.mp4
https://videos.pexels.com/video-files/3413463/3413463-hd_1920_1080_30fps.mp4
3413463-hd_1920_1080_30fps.mp4
https://videos.pexels.com/video-files/9724130/9724130-hd_1440_1080_30fps.mp4
9724130-hd_1440_1080_30fps.mp4


In [ ]:
# Predcit and track the video with the model
for url in urls:
    video_id = extract_video_id(url)
    if video_id:
        hd_link = get_hd_video_link(video_id)
        if hd_link:
            print(f"Proceessing: {hd_link}")
            video_name = get_video_name(hd_link)
            model.track(hd_link, save=True, project=f'{HOME_DIR}/week-06-portfolio/results', conf=0.5, iou=0.9,  tracker="bytetrack.yaml", device=device)
            if os.path.exists(f'{HOME_DIR}/{video_name}'):
                os.remove(f'{HOME_DIR}/{video_name}')
    else:
        print(f"Could not extract video ID from URL: {url}")
        